# Data processing for the Organisational Audit Portfolio spreadsheet 1: rearrange data format

This notebook reads in the "Key Indicators Summary" from the Organisational Audit Portfolio spreadsheet and adjusts the header and index columns to make working with the data more manageable.

In [1]:
import os
import pandas as pd

In [2]:
dir_files = '../data/organisational_audit'
file_excel = '2019-Portfolio.xlsx'

In [3]:
df = pd.read_excel(
    os.path.join(dir_files, file_excel),
    sheet_name='Key Indicators Summary',
    header=[0, 1, 2, 3, 4, 5],
    index_col=[0, 1, 2]
)

In [4]:
df.columns.names

FrozenList(['SCN', 'Trust name', 'Site name', 'Hospital names:', None, None])

In [5]:
df.index.names

FrozenList(['Key indicator', 'Response required to meet indicator', 'National Results'])

### Header: rename missing header names

The header rows can be accessed by index, but it's better to give them names and index by name.

In [6]:
df.columns.names = ['SCN', 'Trust name', 'Site name', 'Hospital names', 'hospital_name_2', 'hospital_name_3']

### Index: remove National Results

Make 'National Results' a normal dataframe column.

In [7]:
df = df.reset_index()

df = df.set_index(['Key indicator', 'Response required to meet indicator'])

In [8]:
df

SCN                                                                                                                                     National Results  \
Trust name                                                                                                                                                 
Site name                                                                                                                                                  
Hospital names                                                                                                                                             
hospital_name_2                                                                                                                                            
hospital_name_3                                                                                                                                            
Key indicator                                      Response required to meet indicator                                                                     
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                 1: 5% (8/169)_x000D_\n2: 7% (12/169)_x000D_\n3...   
Staffing/Workforce                                 NaN                                                 1: 5% (8/169)_x000D_\n2: 7% (12/169)_x000D_\n3...   
1                                                  Key Indicator 1:  Minimum establishment of band...                                       58% (98/169)   
                                                   Band 6 nurses WTE per 10 beds                                      1.9 (1.4-2.9)_x000D_\nMedian (IQR)   
                                                   Band 7 nurses WTE per 10 beds                                      0.5 (0.4-0.8)_x000D_\nMedian (IQR)   
2                                                  Key Indicator 2: Presence of a clinical psychol...                                        7% (12/169)   
                                                   Clinical psychologist WTE per 30 beds (qualified)                    0.1 (0-0.3)_x000D_\nMedian (IQR)   
7-day working                                      Clinical psychologist WTE per 30 beds (qualified)                    0.1 (0-0.3)_x000D_\nMedian (IQR)   
3                                                  Key Indicator 3:  Out of hours presence of stro...                                      71% (101/142)   
                                                   1.7. Do you have stroke specialist nurses (band...                                                      
                                                   Out of hours                                                                            71% (101/142)   
                                                   Registered nurses Type 1 Beds (weekdays 10 pm)                                                    215   
                                                   Registered nurses Type 1 Beds (Saturdays)                                                         243   
                                                   Registered nurses Type 1 Beds (Saturdays 10 pm)                                                   215   
                                                   Registered nurses Type 1 Beds (Sundays)                                                           243   
                                                   Registered nurses Type 1 Beds (Sundays 10 pm)                                                     214   
                                                   Registered nurses Type 3 Beds (weekdays 10 pm)                                                    234   
                                                   Registered nurses Type 3 Beds (Saturdays)                                                         316   
                                                   Registered nurses Type 3 Beds (Saturdays 10 pm)  

### Index: key indicator groups

Convert the index to its own Dataframe to have a better look at it:

In [9]:
df_index = df.index.to_frame(index=False)

df_index

,Key indicator,Response required to meet indicator
0,Total Key Indicators Achieved \n(Post 72hrs si...,NaN
1,Staffing/Workforce,NaN
2,1,Key Indicator 1: Minimum establishment of ban...
3,1,Band 6 nurses WTE per 10 beds
4,1,Band 7 nurses WTE per 10 beds
5,2,Key Indicator 2: Presence of a clinical psycho...
6,2,Clinical psychologist WTE per 30 beds (qualified)
7,7-day working,Clinical psychologist WTE per 30 beds (qualified)
8,3,Key Indicator 3: Out of hours presence of str...
9,3,1.7. Do you have stroke specialist nurses (ban...


Most of the non-integer entries in 'Key indicator' are meant to be another level of multiindexing. They should all have nothing in the 'Response...' column or any other column in the big dataframe.

In [10]:
indicator_groups_dict = {
    'Staffing/Workforce': [1, 2],
    '7-day working': [3, 4, 5],
    'Access to specialist treatment and support': [6, 7],
    'Patient and carer engagement': [8],
    'TIA service': [9],
    'Quality improvement and leadership': [10]
}

In [11]:
# Convert to a dataframe with one row per number:
df_indicator_groups = pd.DataFrame(
    [list(indicator_groups_dict.keys()), list(indicator_groups_dict.values())],
    index=['key_indicator_group', 'Key indicator']
).T.explode('Key indicator', ignore_index=True)

In [12]:
df_indicator_groups

,key_indicator_group,Key indicator
0,Staffing/Workforce,1
1,Staffing/Workforce,2
2,7-day working,3
3,7-day working,4
4,7-day working,5
5,Access to specialist treatment and support,6
6,Access to specialist treatment and support,7
7,Patient and carer engagement,8
8,TIA service,9
9,Quality improvement and leadership,10


Merge this into the existing main dataframe index:

In [13]:
df_index = pd.merge(df_index, df_indicator_groups, on='Key indicator', how='left')

In [14]:
df_index

,Key indicator,Response required to meet indicator,key_indicator_group
0,Total Key Indicators Achieved \n(Post 72hrs si...,NaN,NaN
1,Staffing/Workforce,NaN,NaN
2,1,Key Indicator 1: Minimum establishment of ban...,Staffing/Workforce
3,1,Band 6 nurses WTE per 10 beds,Staffing/Workforce
4,1,Band 7 nurses WTE per 10 beds,Staffing/Workforce
5,2,Key Indicator 2: Presence of a clinical psycho...,Staffing/Workforce
6,2,Clinical psychologist WTE per 30 beds (qualified),Staffing/Workforce
7,7-day working,Clinical psychologist WTE per 30 beds (qualified),NaN
8,3,Key Indicator 3: Out of hours presence of str...,7-day working
9,3,1.7. Do you have stroke specialist nurses (ban...,7-day working


### Index: blank question rows

Some of the entries in the 'Response required...' columns are questions and all other columns have no data for that row. The rows immediately beneath the question row contain possible answers to the question.

Convert this to another multiindex to split off the questions.

In [15]:
question_rows_dict = {
    '1.7. Do you have stroke specialist nurses (band 6 or above) who undertake hyper-acute assessments of suspected stroke patients in A&E?':
        {
            'Out of hours',
            'Registered nurses Type 1 Beds (weekdays 10 pm)',
            'Registered nurses Type 1 Beds (Saturdays)',
            'Registered nurses Type 1 Beds (Saturdays 10 pm)',
            'Registered nurses Type 1 Beds (Sundays)',
            'Registered nurses Type 1 Beds (Sundays 10 pm)',
            'Registered nurses Type 3 Beds (weekdays 10 pm)',
            'Registered nurses Type 3 Beds (Saturdays)',
            'Registered nurses Type 3 Beds (Saturdays 10 pm)',
            'Registered nurses Type 3 Beds (Sundays)',
            'Registered nurses Type 3 Beds (Sundays 10 pm)',
        },
    '1.10. Do the stroke team receive a pre-alert (telephone call) from your ambulance crews for suspected stroke patients?':
        {
            'Thrombolysis candidates only',
            'All FAST positive',
            'All other suspected strokes',
        },
    '1.11. If the stroke team receive a pre-alert, who is the call usually made to? ':
        {
            'Stroke Specialist Nurse',
            'Stroke Junior Doctor on call',
            'Stroke Consultant on call',
        },
    '4.10. Which imaging modality do you most frequently use in your neurovascular clinic for suspected TIAs?':
        {
            '(a) First line brain imaging',
        },
    '7.11. Within what timescale can you see, investigate and initiate treatment for ALL your TIA patients?  ':
        {
            'Outpatient',
            'Outpatient timescale',
        },
    '9.1. What level of management takes responsibility for the follow-up of the results and recommendations of the Sentinel Stroke Audit?':
        {
            'Executive on the Board',
            'Non-executive on the Board',
            'Chairman of Clinical Governance (or equivalent)',
        },
}

In [16]:
# Convert to a dataframe with one row per answer:
df_questions = pd.DataFrame(
    [list(question_rows_dict.keys()), list(question_rows_dict.values())],
    index=['question', 'Response required to meet indicator']
).T.explode('Response required to meet indicator', ignore_index=True)

In [17]:
df_questions

,question,Response required to meet indicator
0,1.7. Do you have stroke specialist nurses (ban...,Registered nurses Type 3 Beds (weekdays 10 pm)
1,1.7. Do you have stroke specialist nurses (ban...,Registered nurses Type 3 Beds (Saturdays 10 pm)
2,1.7. Do you have stroke specialist nurses (ban...,Registered nurses Type 1 Beds (Sundays)
3,1.7. Do you have stroke specialist nurses (ban...,Out of hours
4,1.7. Do you have stroke specialist nurses (ban...,Registered nurses Type 3 Beds (Saturdays)
5,1.7. Do you have stroke specialist nurses (ban...,Registered nurses Type 1 Beds (Sundays 10 pm)
6,1.7. Do you have stroke specialist nurses (ban...,Registered nurses Type 1 Beds (weekdays 10 pm)
7,1.7. Do you have stroke specialist nurses (ban...,Registered nurses Type 3 Beds (Sundays 10 pm)
8,1.7. Do you have stroke specialist nurses (ban...,Registered nurses Type 1 Beds (Saturdays 10 pm)
9,1.7. Do you have stroke specialist nurses (ban...,Registered nurses Type 1 Beds (Saturdays)


In [18]:
# Check that we can pull out these question rows:
mask = df_index['Response required to meet indicator'].isin(df_questions['question'])
df_index['Response required to meet indicator'].loc[mask]

9     1.7. Do you have stroke specialist nurses (ban...
32    1.10. Do the stroke team receive a pre-alert (...
36    1.11. If the stroke team receive a pre-alert, ...
48    4.10. Which imaging modality do you most frequ...
50    7.11. Within what timescale can you see, inves...
55    9.1. What level of management takes responsibi...
Name: Response required to meet indicator, dtype: object

In [19]:
# Check that we can pull out the answer rows:
mask = df_index['Response required to meet indicator'].isin(df_questions['Response required to meet indicator'])
df_index['Response required to meet indicator'].loc[mask]

10                                       Out of hours
11     Registered nurses Type 1 Beds (weekdays 10 pm)
12          Registered nurses Type 1 Beds (Saturdays)
13    Registered nurses Type 1 Beds (Saturdays 10 pm)
14            Registered nurses Type 1 Beds (Sundays)
15      Registered nurses Type 1 Beds (Sundays 10 pm)
16     Registered nurses Type 3 Beds (weekdays 10 pm)
17          Registered nurses Type 3 Beds (Saturdays)
18    Registered nurses Type 3 Beds (Saturdays 10 pm)
19            Registered nurses Type 3 Beds (Sundays)
20      Registered nurses Type 3 Beds (Sundays 10 pm)
33                       Thrombolysis candidates only
34                                  All FAST positive
35                        All other suspected strokes
37                            Stroke Specialist Nurse
38                       Stroke Junior Doctor on call
39                          Stroke Consultant on call
49                       (a) First line brain imaging
51                          

Merge in the question column:

In [20]:
df_index = pd.merge(df_index, df_questions, on='Response required to meet indicator', how='left')

In [21]:
df_index

,Key indicator,Response required to meet indicator,key_indicator_group,question
0,Total Key Indicators Achieved \n(Post 72hrs si...,NaN,NaN,NaN
1,Staffing/Workforce,NaN,NaN,NaN
2,1,Key Indicator 1: Minimum establishment of ban...,Staffing/Workforce,NaN
3,1,Band 6 nurses WTE per 10 beds,Staffing/Workforce,NaN
4,1,Band 7 nurses WTE per 10 beds,Staffing/Workforce,NaN
5,2,Key Indicator 2: Presence of a clinical psycho...,Staffing/Workforce,NaN
6,2,Clinical psychologist WTE per 30 beds (qualified),Staffing/Workforce,NaN
7,7-day working,Clinical psychologist WTE per 30 beds (qualified),NaN,NaN
8,3,Key Indicator 3: Out of hours presence of str...,7-day working,NaN
9,3,1.7. Do you have stroke specialist nurses (ban...,7-day working,NaN


### Whole dataframe: remove unwanted rows based on index

Set the main dataframe index to the combined one we've just made:

In [22]:
df.index = pd.MultiIndex.from_frame(df_index)

Remove the rows of this index that just contain the headings:

In [23]:
bool_rows_to_remove = df.index.get_level_values('Key indicator').isin(list(indicator_groups_dict.keys()))

In [24]:
df = df.loc[~bool_rows_to_remove]

Remove the rows of this index that just contain the questions:

In [25]:
bool_rows_to_remove = df.index.get_level_values('Response required to meet indicator').isin(list(question_rows_dict.keys()))

In [26]:
df = df.loc[~bool_rows_to_remove]

In [27]:
df

SCN                                                                                                                                                                                                                                   National Results  \
Trust name                                                                                                                                                                                                                                               
Site name                                                                                                                                                                                                                                                
Hospital names                                                                                                                                                                                                                                           
hospital_name_2                                                                                                                                                                                                                                          
hospital_name_3                                                                                                                                                                                                                                          
Key indicator                                      Response required to meet indicator                key_indicator_group                        question                                                                                                
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                                        NaN                                                 1: 5% (8/169)_x000D_\n2: 7% (12/169)_x000D_\n3...   
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce                         NaN                                                                                      58% (98/169)   
                                                   Band 6 nurses WTE per 10 beds                      Staffing/Workforce                         NaN                                                                1.9 (1.4-2.9)_x000D_\nMedian (IQR)   
                                                   Band 7 nurses WTE per 10 beds                      Staffing/Workforce                         NaN                                                                0.5 (0.4-0.8)_x000D_\nMedian (IQR)   
2                                                  Key Indicator 2: Presence of a clinical psychol... Staffing/Workforce                         NaN                                                                                       7% (12/169)   
                                                   Clinical psychologist WTE per 30 beds (qualified)  Staffing/Workforce                         NaN                                                                  0.1 (0-0.3)_x000D_\nMedian (IQR)   
3                                                  Key Indicator 3:  Out of hours presence of stro... 7-day working                              NaN                                                                                     71% (101/142)   
                                                   Out of hours                                       7-day working                              1.7. Do you have stroke specialist nurses (band...                                      71% (101/142)   
                                                   Registered nurses Type 1 Beds (weekdays 10 pm)     7-day working                              1.7. Do you have stroke specialist nurses (band...                                                215   


## Save results to file

In [45]:
import numpy as np

def replace_unnamed(df, level, replace_with=''):
    """Change multiindex columns starting 'Unnamed:' to a blank string"""
    old_cols = df.columns.get_level_values(level)
    mask = old_cols.str.startswith('Unnamed')
    new_cols = np.array(old_cols.values)
    new_cols[mask] = replace_with
    df = df.rename(columns=dict(zip(old_cols.tolist(), new_cols)), level=level)
    return df

In [48]:
df = replace_unnamed(df, 'hospital_name_2')
df = replace_unnamed(df, 'hospital_name_3')

In [50]:
df.to_csv('../data/organisational_audit/processed/processed_2019_portfolio_key_indicators_summary.csv')